### 10 Outputs

$ XW = \hat{y} $

$\begin{bmatrix}
  a_1 & b_1\\
  a_2 & b_2 \\
  ... \\
  a_n & b_n
\end{bmatrix} 
\begin{bmatrix}
w_1 ...  w_{10} \\ w_{11} ... w_{20}
\end{bmatrix}
=
\begin{bmatrix}
 y_1 ... y_{10} \\
 ... \\
 y_n ... y_{10n}
\end{bmatrix}
$

### Softmax

$
\sigma(z)_j = \frac{e^{z_j}}{\sum_{k=1}^{K} e^{z_k}} \forall j = 1, ... k
$

Use output as probability, softmax is like sigmoid that fits between 0 / 1, probablilty of output occuring

Takes a linear output first, then softmax transforms the linear output (logit)

Cross entropy is the loss

$D(\hat{y}, y) = -ylog(\hat{y})$

$\alpha = \frac{1}{n} \sum_{i} D(\sigma(wx_i + b), y_i) $ (Sum of loss) (sigma is softmax function)

$\hat{y} = wx_i + b$

In [1]:
import numpy as np

In [2]:
Y = np.array([1, 0, 0])
Y_pred1 = np.array([0.7, 0.2, 0.1]) # generated by the sigma function, softmax values
Y_pred2 = np.array([0.1, 0.3, 0.6])

print(f"loss 1 = {np.sum(-Y * np.log(Y_pred1))}")
print(f"loss 2 = {np.sum(-Y * np.log(Y_pred2))}")

loss 1 = 0.35667494393873245
loss 2 = 2.3025850929940455


In [3]:
import torch as t
import torch.nn as nn

In [4]:
loss = nn.CrossEntropyLoss()

In [5]:
Y = t.Tensor([0]).long() # needs to not be 1 hot but input, 0, 1, or 2, a singular class
Y_pred1 = t.Tensor([[2.0, 1.0, 0.1]])
Y_pred2 = t.Tensor([[0.5, 2.0, 0.3]])
l1 = loss(Y_pred1, Y) # feed logit values directly, cross entropy loss has log softmax in it
l2 = loss(Y_pred2, Y)

In [6]:
print(f"loss 1 = {l1.item()}")
print(f"loss 2 = {l2.item()}")

loss 1 = 0.41703000664711
loss 2 = 1.840616226196289


#### Batch

In [7]:
# classes
Y = t.Tensor([2, 0, 1]).long()
# logits
Y_pred1 = t.Tensor([
    [0.1, 0.2, 0.9],
    [1.1, 0.1, 0.2],
    [0.2, 2.1, 0.1]
])
Y_pred2 = t.Tensor([
    [0.8, 0.2, 0.3],
    [0.2, 0.3, 0.5],
    [0.2, 0.2, 0.5]
])
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)
print(f"loss 1 = {l1.item()}")
print(f"loss 2 = {l2.item()}")

loss 1 = 0.4966353476047516
loss 2 = 1.2388995885849


https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/

https://stackoverflow.com/questions/49390842/cross-entropy-in-pytorch


https://jamesmccaffrey.wordpress.com/2016/09/25/log-loss-and-cross-entropy-are-almost-the-same/

In [8]:
t.nn.Softmax(dim=1)(t.Tensor([[0.1, 0, 0, 1]]))

tensor([[ 0.1898,  0.1717,  0.1717,  0.4668]])

In [9]:
loss(t.Tensor([[0.1, 0, 0, 1]]), t.Tensor([3]).long())

tensor(0.7619)

The linear output [0.1,0,0,1] is transformed into probabilities (sums to 1), where the class dictates which label (0-3) should be the label of an instance, class should be 3 (3rd column), therefore, loss is -1 + log(exp(0.1) + exp(0) + exp(0) + exp(1)) which is = 0.7619

The key idea here is saying, the biggest number leads to the largest log probability, meaning we want the class to match of the column with the biggest number otherwise loss (error) is high, and the other columns to have much smaller numbers than the class column otherwise our confidence is not great

#### Same ways to calculate loss by hand (same result)

Using logits, -1 is the logit of class 3 (3rd column value)

In [10]:
-1 + np.log(np.exp(0.1) + np.exp(0) + np.exp(0) + np.exp(1))

0.7618933412552511

Using probabilty

In [11]:
-1 * np.log(0.4668)

0.7618543785697361

http://willwolf.io/2017/05/18/minimizing_the_negative_log_likelihood_in_english/

BCE for logistic regression (binary probabilities)
CE for softmax (multi class probabilities)

https://pytorch.org/docs/master/nn.html?highlight=nllloss

LogSoftMax + NLLLoss is CrossEntropyLoss

In [31]:
import torch.nn.functional as F

class Net(t.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = x.view(-1, 784) # -1 mean dim is inferred from 28 x 28, flattened
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) # just net outputs

In [32]:
from torchvision import datasets, transforms

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = t.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = t.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [34]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        output = model.forward(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval() # put in eval mode, no gradients
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304789
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.299541


/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296423
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.306162
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.299385
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.298890
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.303989
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.306057
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.304603
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.283331
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.295969
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.303003
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.298502
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.305266
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.301448
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.295375
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.293651
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.293951
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.286144
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.297590
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.306946
Train Epoch: 1 [13440/60000 (22%)]	Loss: 2.286249
Train Epoch: 1 [14080

/Users/achang/miniconda2/envs/deep-learning-lab/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0261, Accuracy: 5546/10000 (55%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.660447
Train Epoch: 2 [640/60000 (1%)]	Loss: 1.581270
Train Epoch: 2 [1280/60000 (2%)]	Loss: 1.474143
Train Epoch: 2 [1920/60000 (3%)]	Loss: 1.419178
Train Epoch: 2 [2560/60000 (4%)]	Loss: 1.381595
Train Epoch: 2 [3200/60000 (5%)]	Loss: 1.268142
Train Epoch: 2 [3840/60000 (6%)]	Loss: 1.273358
Train Epoch: 2 [4480/60000 (7%)]	Loss: 1.019283
Train Epoch: 2 [5120/60000 (9%)]	Loss: 1.079076
Train Epoch: 2 [5760/60000 (10%)]	Loss: 1.134149
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.151120
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.904954
Train Epoch: 2 [7680/60000 (13%)]	Loss: 1.006719
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.875994
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.961699
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.883651
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.890092
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.913467
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.684347
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.357668
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.322899
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.215671
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.276258
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.518217
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.207444
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.211908
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.219625
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.265137
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.479460
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.403197
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.544296
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.209254
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.247154
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.191890
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.446884
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.303397
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.256822
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.515997
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.268607


Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.097395
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.131010
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.159790
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.180660
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.141791
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.244338
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.211598
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.173035
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.089460
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.179199
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.169627
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.201442
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.257080
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.074216
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.372234
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.214672
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.150263
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.278069
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.078030
Train Epoch: 5 [40960/60000 (68%)]	Loss: 0.075992


Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.186456
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.091084
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.140484
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.042278
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.023526
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.092729
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.166856
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.064776
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.104486
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.060486
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.109730
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.113638
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.192605
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.170918
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.192656
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.209279
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.117468
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.063208
Train Epoch: 7 [24960/60000 (42%)]	Loss: 0.158214
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.047568


Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.043191
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.097479
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.140108

Test set: Average loss: 0.0019, Accuracy: 9633/10000 (96%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.031398
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.236865
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.095129
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.125346
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.074211
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.059053
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.148445
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.237028
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.080717
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.060065
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.148942
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.107895
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.033600
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.074096
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.031444
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.019183
Train Epoch: 9 [10

In [35]:
t.save(model.state_dict(), './mnist_model')

#### ToDo Exercise 9-2